In [315]:
import pandas as pd
import numpy as np
import re
import nltk
#from textblob import TextBlob
import json
import matplotlib.pyplot as plt
import csv



In [316]:
#load data from csv/excel/json
data_ref=pd.read_excel('tweets.xlsx') 
#data=pd.read_json('dataset_filt.jsonl')

#data.head()

In [317]:
data=data_ref

#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    #text = text.lower().replace("ё", "е")
    text = text.replace("ё", "е")
    dat_str=data_ref.iloc[0]['full_text']
    text = re.sub(dat_str[-10:-7],'\'', text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    text = re.sub(' +',' ', text)
    text = re.sub(r' &amp','\'', text)
    text = re.sub(r'\r','', text)
    text = re.sub(r'\n','', text)
    text=deEmojify(text)
    #print(text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    #text = re.sub(' +',' ', text)
    return text.strip()


#print(data.iloc[0]['full_text'])
#text = [preprocess_text(t) for t in data.iloc[:]['full_text']]
#storing preprocessed texts in a separate column 
#data["processed_text"]= text
#print(data.iloc[0]["processed_text"])
#print(dat_str[-10:-7])

In [318]:
#Vader sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
#calling the Textblob function for each text

cnt=0

def write_json(data, filename='Tweet_Sentiment.json'): 
    with open(filename,'a') as f: 
        json.dump(data, f, indent=4) 
        
wtr = csv.writer(open('Tweet_sentiment_Vader.csv', 'w'), delimiter=',', lineterminator='\n')   
with open('dataset_filt.jsonl') as json_file: 

       for line in json_file:
                cnt=cnt+1
                
                data=(json.loads(line))
                
                text = preprocess_text(data['full_text']) 
                
                
                polarity=analyser.polarity_scores(text)

                #df["Polarity/subjectivity"]= polarity

                
               
                if polarity['compound']>0.05:
                        sentiment="Positive"
                elif polarity['compound']<-0.05:
                        sentiment="Negative"
                else:
                        sentiment="Neutral"
                
                #df = pd.DataFrame(np.reshape([data['id_str'],,sentiment],(1,3)))
                #df.to_csv('Tweet_sentiment_Vader.csv',mode='a')
                wtr.writerow([data['id_str']+",",data['full_text'],","+sentiment])
                x =  { "id_str":data['id_str'],"full_text":data['full_text'],"Sentiment":sentiment}
                
                write_json(x)  
                
                
                
#df[["processed_text","Sentiment"]]           